# Simulating Boids Algorithm


Includes code on Boids Algorithm and saving that information over time.


In [3]:
import pygame
import numpy as np
import random
import math
import pandas as pd
import os

# Constants
WIDTH, HEIGHT = 1000, 1000
NUM_BOIDS = 100
VISUAL_RANGE = 75
DRAW_TRAIL = False

# Colors
WHITE = (255, 255, 255)
BOID_COLOR = (85, 140, 244)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Boid Simulation")

# Initialize Boids
boids = []

def initBoids():
    global boids
    boids = []
    for _ in range(NUM_BOIDS):
        boids.append({
            'x': random.random() * WIDTH,
            'y': random.random() * HEIGHT,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })
    return pd.DataFrame(boids)

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def flyTowardsCenter(boid):
    centeringFactor = 0.005
    centerX = 0
    centerY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < VISUAL_RANGE:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1
    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors
        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0
    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']
    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05
    avgDX = 0
    avgDY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < VISUAL_RANGE:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1
    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors
        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1
    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > WIDTH - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > HEIGHT - margin:
        boid['dy'] -= turnFactor

def limitSpeed(boid):
    speedLimit = 10
    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, BOID_COLOR, [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)
    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, BOID_COLOR, (int(point[0]), int(point[1])), 1)

def animationLoop():
    global boids
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)
        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]
    screen.fill(WHITE)
    for boid in boids:
        drawBoid(screen, boid)
    pygame.display.flip()
    pygame.time.Clock().tick(60)

# Initialize Boids and Main Loop
initBoids()
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    animationLoop()

pygame.quit()


KeyboardInterrupt: 

## Running Boids Algorithm and Saving Information Into Pandas DataFrame


In [4]:
directory = '../data/myjson'
if not os.path.exists(directory):
   os.makedirs(directory)

In [12]:
num_sims = 1
num_time_steps = 10000

# Initialize final_df as an empty DataFrame
final_df = pd.DataFrame()

for n in range(num_sims):  # Run simulation num_sims times
    curr_df_list = []  # List to store DataFrames for current simulation
    curr_df = initBoids()  # Initialize boids with different positions and velocities
    curr_df['Simulation'] = [n] * len(curr_df)  # Set Simulation number
    curr_df['Timestep'] = [0] * len(curr_df)  # Set Timestep number to 0
    curr_df_list.append(curr_df)  # Append initial state to list

    for t in range(1, num_time_steps):  # Run animationLoop() for num_time_steps
        new_df = animationLoop()  # Updates boids to have new positions and velocities
        new_df['Simulation'] = [n] * len(new_df)  # Set Simulation number
        new_df['Timestep'] = [t] * len(new_df)  # Set Timestep number
        curr_df_list.append(new_df)  # Append updated state to list

    # Concatenate all DataFrames in the list once per simulation
    curr_df = pd.concat(curr_df_list, ignore_index=True)
    # Append the result of the current simulation to final_df
    final_df = pd.concat([final_df, curr_df], ignore_index=True)

# Reset the index of the final DataFrame
final_df.reset_index(drop=True, inplace=True)

# Save the final DataFrame to a JSON file
final_df.to_json(os.path.join(directory, 'boid_simulation_data.json'), orient='records')

print(new_df)

KeyboardInterrupt: 

In [39]:
final_df

,x,y,dx,dy,history,Simulation,Timestep
0,301.456894,81.482953,-0.104859,-1.745531,"[(301.42536424632004, 80.6175761175713)]",0,0
1,324.236434,549.970175,0.609287,-2.401889,"[(324.77970282470403, 547.6960821908741)]",0,0
2,897.566370,585.511241,-3.785262,-0.529240,"[(892.9221803741805, 585.0661879678225)]",0,0
3,16.398136,222.909364,0.655671,-2.525104,"[(18.054752966270854, 220.61965793939558)]",0,0
4,499.756969,409.117480,2.581771,2.058534,"[(502.36005525021574, 411.0076723848969)]",0,0
...,...,...,...,...,...,...,...
999995,740.097688,302.411541,1.021128,-9.947728,"[(796.830502848903, 770.0311137509005), (796.5...",0,9999
999996,782.497615,350.772573,-0.011763,-9.984775,"[(772.9322117499539, 833.880922975768), (770.9...",0,9999
999997,535.913864,291.296185,-2.127204,-9.771131,"[(586.8395839875654, 741.1429645854976), (587....",0,9999
999998,621.977637,274.696146,-0.448837,-9.531054,"[(628.7723301005842, 756.2598828376217), (630....",0,9999


## Converting Simulation DataFrame to CSV


In [32]:
path_to_save = 'simulation.csv'
final_df.to_csv(path_to_save, index=False)


## Save Edges From Each Timestep/Simulation


In [40]:
path_to_save = '../data/simulation.csv'
final_df = pd.read_csv(path_to_save)


### Compute The Edges Per Timestep Per Simulation


In [41]:
from scipy.spatial.distance import pdist, squareform
# Distance threshold


# Function to calculate pairwise distances and return edges
def get_edges(df, threshold):
  
   distances = squareform(pdist(df[['x', 'y']])) # Calculate pairwise distances
  
   close_pairs = distances < threshold # Identify pairs within the threshold distance
  
   # Extract indices of close pairs
   edges = [(i, j) for i in range(len(distances)) for j in range(i+1, len(distances)) if close_pairs[i, j]]
  
   # Create DataFrame for edges
   edges_df = pd.DataFrame(edges, columns=['Boid_i', 'Boid_j'])
  
   return edges_df




edges_dfs = []  # List to collect DataFrames
final_df_groupby = final_df.groupby(['Timestep', 'Simulation'])
for key, item in final_df_groupby:
   edges_df = get_edges(item, visualRange)  # Pass the group directly
   edges_df['Timestep'] = key[0]
   edges_df['Simulation'] = key[1]
   edges_dfs.append(edges_df)


# Concatenate all DataFrames at once, if edges_dfs is not empty
if edges_dfs:
   final_edges_df = pd.concat(edges_dfs, ignore_index=True)
   final_edges_df.reset_index(drop=True, inplace=True)  # Reset index once, outside the loop
else:
   final_edges_df = pd.DataFrame()  # Initialize to an empty DataFrame if no edges were found


final_edges_df


,Boid_i,Boid_j,Timestep,Simulation
0,0,23,0,0
1,0,64,0,0
2,1,12,0,0
3,2,72,0,0
4,3,15,0,0
...,...,...,...,...
721582,92,95,1126,0
721583,0,4,1127,0
721584,0,7,1127,0
721585,4,7,1127,0


### Save final_edges_df As A CSV


In [42]:
path_to_save = '../data/simulation_edges.csv'
final_edges_df.to_csv(path_to_save, index=False)


### Test

In [45]:
import pygame
import numpy as np
import random
import math
import pandas as pd
import json
import os

# Constants
WIDTH, HEIGHT = 1000, 1000
NUM_BOIDS = 100
VISUAL_RANGE = 75
DRAW_TRAIL = False
FRAMES_PER_FILE = 1000
NUM_FILES = 10

# Colors
WHITE = (255, 255, 255)
BOID_COLOR = (85, 140, 244)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Boid Simulation")

# Initialize Boids
boids = []
data = []  # For storing boid data over time

def initBoids():
    global boids
    boids = []
    for _ in range(NUM_BOIDS):
        boids.append({
            'x': random.random() * WIDTH,
            'y': random.random() * HEIGHT,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def flyTowardsCenter(boid):
    centeringFactor = 0.005
    centerX = 0
    centerY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < VISUAL_RANGE:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1
    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors
        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0
    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']
    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05
    avgDX = 0
    avgDY = 0
    numNeighbors = 0
    for otherBoid in boids:
        if distance(boid, otherBoid) < VISUAL_RANGE:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1
    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors
        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1
    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > WIDTH - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > HEIGHT - margin:
        boid['dy'] -= turnFactor

def limitSpeed(boid):
    speedLimit = 10
    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, BOID_COLOR, [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)
    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, BOID_COLOR, (int(point[0]), int(point[1])), 1)

def animationLoop():
    global boids, data
    frame_data = []
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)
        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

        # Collect data for this frame
        frame_data.append({'x': boid['x'], 'y': boid['y'], 'dx': boid['dx'], 'dy': boid['dy']})
    
    data.append(frame_data)  # Append this frame's data to the global data list
    screen.fill(WHITE)
    for boid in boids:
        drawBoid(screen, boid)
    pygame.display.flip()
    pygame.time.Clock().tick(60)

# Initialize Boids and Main Loop
initBoids()
running = True
frame_count = 0
file_count = 0
json_folder = 'boid_data'
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

while running and file_count < NUM_FILES:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    animationLoop()
    frame_count += 1

    if frame_count >= FRAMES_PER_FILE:
        with open(os.path.join(json_folder, f'boid_data_{file_count}.json'), 'w') as f:
            json.dump(data, f)
        print(f"boid_data_{file_count}.json file has been saved.")
        data = []  # Reset data for next file
        frame_count = 0
        file_count += 1

pygame.quit()
if data:  # Save any remaining data
    with open(os.path.join(json_folder, f'boid_data_{file_count}.json'), 'w') as f:
        json.dump(data, f)
    print(f"boid_data_{file_count}.json file has been saved.")



boid_data_0.json file has been saved.
boid_data_1.json file has been saved.
boid_data_2.json file has been saved.
boid_data_3.json file has been saved.
boid_data_4.json file has been saved.
boid_data_5.json file has been saved.
boid_data_6.json file has been saved.
boid_data_7.json file has been saved.
boid_data_8.json file has been saved.
boid_data_9.json file has been saved.


In [46]:
import pandas as pd
import json
import os

# Define the folder and file pattern
json_folder = 'boid_data'
json_files = [os.path.join(json_folder, f'boid_data_{i}.json') for i in range(NUM_FILES)]

# Load JSON files and concatenate into a single DataFrame
df_list = []
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
        for frame in data:
            df_list.append(pd.DataFrame(frame))

# Concatenate all DataFrames
df = pd.concat(df_list, ignore_index=True)
df.to_csv('boid_data.csv', index=False)
print('Data saved to boid_data.csv')


Data saved to boid_data.csv


In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Load the data
df = pd.read_csv('boid_data.csv')

# Normalize the data
scaler = MinMaxScaler()
df[['x', 'y', 'dx', 'dy']] = scaler.fit_transform(df[['x', 'y', 'dx', 'dy']])

# Prepare the data for RNN
sequence_length = 10
X = []
y = []

for i in range(len(df) - sequence_length):
    X.append(df[['x', 'y', 'dx', 'dy']].values[i:i+sequence_length])
    y.append(df[['x', 'y']].values[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create DataLoader
dataset


: 